This isn't part of the automated pipeline, but needed to get input data ready. It may vary depending on the data source.


# Extract and merge Gitter data

In [ ]:
import os
import zipfile
from doctest import debug

import pandas as pd
import re


# To use for all three data levels, find and replace '100m' with '1km' or '10km' as needed.

def extract_zip_files(zip_folder, extract_to):
    """Extracts all zip files in the given folder."""
    zip_files = [f for f in os.listdir(zip_folder) if f.endswith(".zip")]
    print(f"Found {len(zip_files)} zip files.")

    extracted_files = []
    for zip_file in zip_files:
        zip_path = os.path.join(zip_folder, zip_file)
        with zipfile.ZipFile(zip_path, 'r') as z:
            z.extractall(extract_to)
            extracted_files.extend([os.path.join(extract_to, name) for name in z.namelist()])

    print(f"Extracted {len(extracted_files)} files.")
    return extracted_files


def find_csv_files(files):
    """Finds CSV files that contain '1km-Gitter' in the name."""
    csv_files = [f for f in files if "1km-Gitter" in f and f.endswith(".csv")]
    print(f"Found {len(csv_files)} relevant CSV files.")
    return csv_files


def clean_text(text):
    """Removes special characters and replaces them with an empty string."""
    return re.sub(r'[^\x20-\x7E]', '', text)  # Keeps only printable ASCII characters


def convert_numeric_columns(df):
    """Ensures numeric columns with comma decimals ('9,07') are properly converted to floats."""
    for col in df.columns:
        # Only modify string columns
        if df[col].dtype == object:
            df[col] = df[col].str.replace(',', '.', regex=False)  # Replace commas with dots for decimal conversion
            df[col] = pd.to_numeric(df[col], errors='ignore')  # Convert to float, set invalid entries to NaN

    return df


def read_csv_with_encoding(file):
    """Reads CSV with UTF-8 first, falls back to ISO-8859-1 if needed."""
    try:
        df = pd.read_csv(file, encoding='utf-8', dtype=str, sep=';', encoding_errors='replace')
    except UnicodeDecodeError:
        print(f"Warning: UTF-8 failed for {file}, trying ISO-8859-1.")
        df = pd.read_csv(file, encoding='ISO-8859-1', dtype=str, sep=';', encoding_errors='replace')

    # Apply text cleaning to all string values
    df = df.applymap(lambda x: clean_text(x) if isinstance(x, str) else x)

    # Convert numeric values from '9,07' to 9.07
    df = convert_numeric_columns(df)

    return df


def process_and_merge_csv(files):
    """Reads, processes, and merges CSV files on 'GITTER_ID_1km'."""
    merged_df = None

    for file in files:
        print(f"Processing: {file}")
        df = read_csv_with_encoding(file)

        # Standardize column names
        df.columns = df.columns.str.strip()

        # Drop unwanted columns if they exist
        df.drop(columns=[col for col in ['x_mp_1km', 'y_mp_1km'] if col in df.columns], inplace=True, errors='ignore')

        if 'GITTER_ID_1km' not in df.columns:
            print(f"Warning: 'GITTER_ID_1km' not found in {file}. Available columns: {df.columns.tolist()}")
            continue

        # Rename duplicate columns before merging
        df = df.rename(columns={col: f"{col}_{os.path.basename(file).split('.')[0]}" for col in df.columns if
                                col != 'GITTER_ID_1km'})

        if merged_df is None:
            merged_df = df
        else:
            merged_df = merged_df.merge(df, on='GITTER_ID_1km', how='outer', suffixes=(None, None))

    print("Merging complete.")
    return merged_df


# Define paths
zip_folder = r"C:\Users\petre\Desktop\Synpop\Zensus"
extract_to = r"C:\Users\petre\Desktop\Synpop\Zensus\Extrahiert"
output_file = "merged_1km_gitter.csv"
output_excel = "merged_1km_gitter.xlsx"

# Execution pipeline
extracted_files = extract_zip_files(zip_folder, extract_to)
csv_files = find_csv_files(extracted_files)
merged_data = process_and_merge_csv(csv_files)

# Save to CSV
if merged_data is not None:
    merged_data.to_csv(output_file, index=False)
    print(f"Saved merged file to {output_file}")
else:
    print("No valid data to save.")

# # Save to Excel -> sheet to large
# if merged_data is not None:
#     merged_data.to_excel(output_excel, index=False)
#     print(f"Saved merged file to {output_excel}")


# Impute missing/inaccurate population values (just multiplication to fit the total -> very low values stay very low)

In [38]:
import pandas as pd
import re


def normalize_id(gid, replace_digits):
    match = re.search(r'(N\d+)(E\d+)', gid)
    if match:
        n_part, e_part = match.groups()
        n_base = n_part[:-replace_digits] + '0' * replace_digits
        e_base = e_part[:-replace_digits] + '0' * replace_digits
        return n_base + e_base
    return gid


def smart_integerize(df, pop_cols):
    for pop_col in pop_cols:
        total_before = df[pop_col].sum()
        df[pop_col] = df[pop_col].round().astype(int)
        total_after = df[pop_col].sum()
        diff = total_before - total_after
        if diff != 0:
            sorted_indices = df[pop_col].abs().sort_values(ascending=False).index[:abs(int(diff))]
            df.loc[sorted_indices, pop_col] += int(diff / abs(diff))


def adjust_population(lower_level_file, higher_level_file, replace_digits_lower, replace_digits_higher, id_col_lower,
                      id_col_higher, pop_cols_lower, pop_cols_higher, output_file):
    lower_df = pd.read_csv(lower_level_file)
    higher_df = pd.read_csv(higher_level_file)

    assert id_col_lower in lower_df.columns and all(
        col in lower_df.columns for col in pop_cols_lower), "Lower level dataset missing required columns."
    assert id_col_higher in higher_df.columns and all(
        col in higher_df.columns for col in pop_cols_higher), "Higher level dataset missing required columns."

    lower_df['GroupID'] = lower_df[id_col_lower].apply(lambda x: normalize_id(str(x), replace_digits_lower))
    higher_df['GroupID'] = higher_df[id_col_higher].apply(lambda x: normalize_id(str(x), replace_digits_higher))

    lower_grouped = lower_df.groupby('GroupID')[pop_cols_lower].sum().reset_index()
    merged = lower_grouped.merge(higher_df[['GroupID'] + pop_cols_higher], on='GroupID', how='left')

    for pop_lower, pop_higher in zip(pop_cols_lower, pop_cols_higher):
        merged[pop_higher] = merged[pop_higher].fillna(merged[pop_lower])
        merged[f'AdjustmentFactor_{pop_higher}'] = merged[pop_higher] / merged[pop_lower]
        merged[f'AdjustmentFactor_{pop_higher}'].fillna(1.0, inplace=True)

    lower_df = lower_df.merge(merged[['GroupID'] + [f'AdjustmentFactor_{col}' for col in pop_cols_higher]],
                              on='GroupID', how='left')

    for pop_lower, pop_higher in zip(pop_cols_lower, pop_cols_higher):
        lower_df[f'Adjusted_{pop_lower}'] = lower_df[pop_lower] * lower_df[f'AdjustmentFactor_{pop_higher}']

    # Set nans to 0
    lower_df.fillna(0, inplace=True)
    # This ensures total population, but within the cell, it may cause minor differences (since this is global)
    smart_integerize(lower_df, [f'Adjusted_{col}' for col in pop_cols_lower])

    save_csv = lower_df[[id_col_lower] + [f'Adjusted_{col}' for col in pop_cols_lower]]
    # Remove the adjusted prefix in the col name
    save_csv.columns = [col.replace('Adjusted_', '') for col in save_csv.columns]
    save_csv.to_csv(output_file, index=False)

    print(f"Adjusted population data saved to {output_file}")


adjust_population(r"C:\Users\petre\Documents\GitHub\MATSimPipeline\data\syn_pop\merged_1km_gitter.csv",
                  r"C:\Users\petre\Documents\GitHub\MATSimPipeline\data\syn_pop\merged_10km_gitter.csv",
                  replace_digits_lower=4, replace_digits_higher=4,
                  id_col_lower='GITTER_ID_1km', id_col_higher='GITTER_ID_10km',
                  pop_cols_lower=['Insgesamt_Bevoelkerung_Zensus2022_Alter_in_10er-Jahresgruppen_1km-Gitter',
                                  'Insgesamt_Bevoelkerung_Zensus2022_Alter_in_5_Altersklassen_1km-Gitter'],
                  pop_cols_higher=['Insgesamt_Bevoelkerung_Zensus2022_Alter_in_10er-Jahresgruppen_10km-Gitter',
                                   'Insgesamt_Bevoelkerung_Zensus2022_Alter_in_5_Altersklassen_10km-Gitter'],
                  output_file="1adjusted_1km.csv")

adjust_population(r"C:\Users\petre\Documents\GitHub\MATSimPipeline\data\syn_pop\merged_100m_gitter.csv",
                  "1adjusted_1km.csv", replace_digits_lower=3, replace_digits_higher=3,
                  id_col_lower='GITTER_ID_100m', id_col_higher='GITTER_ID_1km',
                  pop_cols_lower=['Insgesamt_Bevoelkerung_Zensus2022_Alter_in_10er-Jahresgruppen_100m-Gitter',
                                  'Insgesamt_Bevoelkerung_Zensus2022_Alter_5Altersklassen_100m-Gitter'],
                  pop_cols_higher=['Insgesamt_Bevoelkerung_Zensus2022_Alter_in_10er-Jahresgruppen_1km-Gitter',
                                   'Insgesamt_Bevoelkerung_Zensus2022_Alter_in_5_Altersklassen_1km-Gitter'],
                  output_file="1adjusted_100m.csv")


C:\Users\petre\AppData\Local\Temp\ipykernel_15232\3029198941.py:24: DtypeWarning: Columns (27,34,36) have mixed types. Specify dtype option on import or set low_memory=False.
  lower_df = pd.read_csv(lower_level_file)
C:\Users\petre\AppData\Local\Temp\ipykernel_15232\3029198941.py:39: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged[f'AdjustmentFactor_{pop_higher}'].fillna(1.0, inplace=True)
C:\Users\petre\AppData\Local\Temp\ipykernel_15232\3029198941.py:39: FutureWarning: A value is trying to be set on a copy of a DataFra

Adjusted population data saved to adjusted_100m.csv


# Merge and do some cleanup

In [42]:
import pandas as pd


def merge_adjusted_with_original(original_file, adjusted_file, pop_cols, to_file):
    original_df = pd.read_csv(original_file)
    adjusted_df = pd.read_csv(adjusted_file)

    for pop_col in pop_cols:
        total_diff = adjusted_df[pop_col].sum() - original_df[pop_col].sum()
        max_diff = (adjusted_df[pop_col] - original_df[pop_col]).abs().max()
        print(f"{pop_col} - Total Difference: {total_diff}, Max Difference per row: {max_diff}")

    for pop_col in pop_cols:
        original_df[f'adjustment_diff_{pop_col}'] = adjusted_df[pop_col] - original_df[pop_col]
        original_df[pop_col] = adjusted_df[pop_col]

    if "Insgesamt_Bevoelkerung_Zensus2022_Alter_5Altersklassen_100m-Gitter" in original_df.columns:
        original_df.rename(columns={
            "Insgesamt_Bevoelkerung_Zensus2022_Alter_5Altersklassen_100m-Gitter": "Insgesamt_Bevoelkerung_Zensus2022_Alter_in_5_Altersklassen_100m-Gitter"},
            inplace=True)

    original_df.to_csv(to_file, index=False)
    print(f"Merged data saved to {to_file}")


merge_adjusted_with_original(r"C:\Users\petre\Documents\GitHub\MATSimPipeline\data\syn_pop\merged_1km_gitter.csv",
                             "adjusted_1km.csv", [
                                 'Insgesamt_Bevoelkerung_Zensus2022_Alter_in_10er-Jahresgruppen_1km-Gitter',
                                 'Insgesamt_Bevoelkerung_Zensus2022_Alter_in_5_Altersklassen_1km-Gitter'
                             ],
                             "2poptotal_adjusted_1km_gitter.csv")

merge_adjusted_with_original(r"C:\Users\petre\Documents\GitHub\MATSimPipeline\data\syn_pop\merged_100m_gitter.csv",
                             "adjusted_100m.csv", [
                                 'Insgesamt_Bevoelkerung_Zensus2022_Alter_in_10er-Jahresgruppen_100m-Gitter',
                                 'Insgesamt_Bevoelkerung_Zensus2022_Alter_5Altersklassen_100m-Gitter'
                             ],
                             "2poptotal_adjusted_100m_gitter.csv")

C:\Users\petre\AppData\Local\Temp\ipykernel_15232\967282453.py:4: DtypeWarning: Columns (34) have mixed types. Specify dtype option on import or set low_memory=False.
  original_df = pd.read_csv(original_file)


Insgesamt_Bevoelkerung_Zensus2022_Alter_in_10er-Jahresgruppen_1km-Gitter - Total Difference: 4921.0, Max Difference per row: 13.0
Insgesamt_Bevoelkerung_Zensus2022_Alter_in_5_Altersklassen_1km-Gitter - Total Difference: 4921.0, Max Difference per row: 13.0
Merged data saved to poptotal_adjusted_1km_gitter.csv


C:\Users\petre\AppData\Local\Temp\ipykernel_15232\967282453.py:4: DtypeWarning: Columns (27,34,36) have mixed types. Specify dtype option on import or set low_memory=False.
  original_df = pd.read_csv(original_file)


Insgesamt_Bevoelkerung_Zensus2022_Alter_in_10er-Jahresgruppen_100m-Gitter - Total Difference: 135944.0, Max Difference per row: 12.0
Insgesamt_Bevoelkerung_Zensus2022_Alter_5Altersklassen_100m-Gitter - Total Difference: 135944.0, Max Difference per row: 12.0
Merged data saved to poptotal_adjusted_100m_gitter.csv


# Ensure the population totals match

In [45]:
compare1 = pd.read_csv("2poptotal_adjusted_1km_gitter.csv")
compare2 = pd.read_csv("2poptotal_adjusted_100m_gitter.csv")
diff = compare1["Insgesamt_Bevoelkerung_Zensus2022_Alter_in_10er-Jahresgruppen_1km-Gitter"] - compare1[
    "Insgesamt_Bevoelkerung_Zensus2022_Alter_in_5_Altersklassen_1km-Gitter"]
print(max(diff))
diff = compare2["Insgesamt_Bevoelkerung_Zensus2022_Alter_in_10er-Jahresgruppen_100m-Gitter"] - compare2[
    "Insgesamt_Bevoelkerung_Zensus2022_Alter_in_5_Altersklassen_100m-Gitter"]
print(max(diff))

C:\Users\petre\AppData\Local\Temp\ipykernel_15232\2574285654.py:1: DtypeWarning: Columns (34) have mixed types. Specify dtype option on import or set low_memory=False.
  compare1 = pd.read_csv("poptotal_adjusted_1km_gitter.csv")
C:\Users\petre\AppData\Local\Temp\ipykernel_15232\2574285654.py:2: DtypeWarning: Columns (27,34,36) have mixed types. Specify dtype option on import or set low_memory=False.
  compare2 = pd.read_csv("poptotal_adjusted_100m_gitter.csv")


0
0


# Impute age distribution

In [31]:
# Add higher level IDs to 1km and 100m

def normalize_id(gid, replace_digits):
    match = re.search(r'(N\d+)(E\d+)', gid)
    if match:
        n_part, e_part = match.groups()
        n_base = n_part[:-replace_digits] + '0' * replace_digits
        e_base = e_part[:-replace_digits] + '0' * replace_digits
        return n_base + e_base
    return gid


def add_higher_level_id(df, id_col, replace_digits, new_col_name):
    df[new_col_name] = df[id_col].apply(lambda x: normalize_id(str(x), replace_digits))
    return df


def prepend_id(df, id_col, prefix, new_col_name):
    df[new_col_name] = prefix + df[id_col].astype(str)
    return df


# Add 10km ID to 1km
one_km_df = pd.read_csv("2poptotal_adjusted_1km_gitter.csv")
one_km_df = add_higher_level_id(one_km_df, 'GITTER_ID_1km', 4, 'GITTER_ID_10km')
one_km_df = prepend_id(one_km_df, 'GITTER_ID_10km', 'CRS3035RES10000m', 'GITTER_ID_10km')
one_km_df.to_csv("3adjusted_1km_with_higher.csv", index=False)

# Add 1km ID to 100m
hundred_m_df = pd.read_csv("2poptotal_adjusted_100m_gitter.csv")
hundred_m_df = add_higher_level_id(hundred_m_df, 'GITTER_ID_100m', 3, 'GITTER_ID_1km')
hundred_m_df = prepend_id(hundred_m_df, 'GITTER_ID_1km', 'CRS3035RES1000m', 'GITTER_ID_1km')

# Add 10km ID to 100m
hundred_m_df = add_higher_level_id(hundred_m_df, 'GITTER_ID_100m', 4, 'GITTER_ID_10km')
hundred_m_df = prepend_id(hundred_m_df, 'GITTER_ID_10km', 'CRS3035RES10000m', 'GITTER_ID_10km')
hundred_m_df.to_csv("3adjusted_100m_with_higher.csv", index=False)


C:\Users\petre\AppData\Local\Temp\ipykernel_17900\3198254823.py:21: DtypeWarning: Columns (34) have mixed types. Specify dtype option on import or set low_memory=False.
  one_km_df = pd.read_csv("2poptotal_adjusted_1km_gitter.csv")
C:\Users\petre\AppData\Local\Temp\ipykernel_17900\3198254823.py:27: DtypeWarning: Columns (27,34,36) have mixed types. Specify dtype option on import or set low_memory=False.
  hundred_m_df = pd.read_csv("2poptotal_adjusted_100m_gitter.csv")


In [4]:
import pandas as pd
import numpy as np
import re


def smart_integerize(df, pop_cols):
    for pop_col in pop_cols:
        total_before = df[pop_col].sum()
        df[pop_col] = df[pop_col].round().astype(int)
        total_after = df[pop_col].sum()
        diff = total_before - total_after
        if diff != 0:
            sorted_indices = df[pop_col].abs().sort_values(ascending=False).index[:abs(int(diff))]
            df.loc[sorted_indices, pop_col] += int(diff / abs(diff))
    return df


def ipf_adjustment(df, row_control_totals, col_control_totals, age_cols, max_iterations=10, tol=1e-6):
    for _ in range(max_iterations):
        
        # We do col total matching last here, because we are interested, at 10km in the best age DISTRIBUTION, not exact matching total.
        row_totals = df[age_cols].sum(axis=1)
        scaling_factors = row_control_totals / row_totals
        # Set inf to 1
        scaling_factors[~np.isfinite(scaling_factors)] = 1
        df[age_cols] = df[age_cols].multiply(scaling_factors, axis=0)

        col_totals = df[age_cols].sum()
        scaling_factors = col_control_totals / col_totals
        # Set inf to 1
        scaling_factors[~np.isfinite(scaling_factors)] = 1
        df[age_cols] = df[age_cols].multiply(scaling_factors, axis=1)
        print(df[age_cols].sum().sum())
        error = 1
        if error < tol:
            break
    return df


def mix_age_distribution(df, age_cols, total_col, reference_distribution, threshold=150):
    cell_count = 0
    for index, row in df.iterrows():
        pop = row[total_col]
        if threshold > pop > 0:
            cell_count += 1
            weight = pop / threshold
            adjusted_values = weight * row[age_cols] + (1 - weight) * reference_distribution * pop
            df.loc[index, age_cols] = adjusted_values  #/ adjusted_values.sum() * pop
    print(f"Adjusted {cell_count} cells.")
    print(f"Total cells: {len(df)}")
    return df


# Load preprocessed data
#one_km_df = pd.read_csv("3adjusted_1km_with_higher.csv")
#hundred_m_df = pd.read_csv("3adjusted_100m_with_higher.csv")
ten_km_df = pd.read_csv(r"C:\Users\petre\Documents\GitHub\MATSimPipeline\data\syn_pop\merged_10km_gitter.csv")

# Adjust the 10km level using IPF and mix age distribution
print("Adjusting age distribution at the 10km level...")
age_cols_10km_10er = [
    # 10er-Gruppen
    'Unter10_Zensus2022_Alter_in_10er-Jahresgruppen_10km-Gitter',
    'a10bis19_Zensus2022_Alter_in_10er-Jahresgruppen_10km-Gitter',
    'a20bis29_Zensus2022_Alter_in_10er-Jahresgruppen_10km-Gitter',
    'a30bis39_Zensus2022_Alter_in_10er-Jahresgruppen_10km-Gitter',
    'a40bis49_Zensus2022_Alter_in_10er-Jahresgruppen_10km-Gitter',
    'a50bis59_Zensus2022_Alter_in_10er-Jahresgruppen_10km-Gitter',
    'a60bis69_Zensus2022_Alter_in_10er-Jahresgruppen_10km-Gitter',
    'a70bis79_Zensus2022_Alter_in_10er-Jahresgruppen_10km-Gitter',
    'a80undaelter_Zensus2022_Alter_in_10er-Jahresgruppen_10km-Gitter']

age_cols_10km_5er = [
    # 5er-Gruppen
    'Unter18_Zensus2022_Alter_in_5_Altersklassen_10km-Gitter',
    'a18bis29_Zensus2022_Alter_in_5_Altersklassen_10km-Gitter',
    'a30bis49_Zensus2022_Alter_in_5_Altersklassen_10km-Gitter',
    'a50bis64_Zensus2022_Alter_in_5_Altersklassen_10km-Gitter',
    'a65undaelter_Zensus2022_Alter_in_5_Altersklassen_10km-Gitter'
]
# Count nans
nans = ten_km_df[age_cols_10km_10er+age_cols_10km_5er].isna().sum().sum()
print(f"Total nans: {nans}")
# Set nans in given cols to 0
ten_km_df[age_cols_10km_10er+age_cols_10km_5er] = ten_km_df[age_cols_10km_10er+age_cols_10km_5er].fillna(0)
print("Nans set to 0.")

reference_distribution_10km_10er = ten_km_df[age_cols_10km_10er].sum() / ten_km_df[age_cols_10km_10er].sum().sum()
reference_distribution_10km_5er = ten_km_df[age_cols_10km_5er].sum() / ten_km_df[age_cols_10km_5er].sum().sum()
col_totals_10er = ten_km_df[age_cols_10km_10er].sum().copy()
col_totals_5er = ten_km_df[age_cols_10km_5er].sum().copy()
row_totals_10er = ten_km_df['Insgesamt_Bevoelkerung_Zensus2022_Alter_in_10er-Jahresgruppen_10km-Gitter'].copy()
row_totals_5er = ten_km_df['Insgesamt_Bevoelkerung_Zensus2022_Alter_in_5_Altersklassen_10km-Gitter'].copy()
print(f"Diff in totals (should be 0): {row_totals_10er.sum() - row_totals_5er.sum()}")
ten_km_df = mix_age_distribution(ten_km_df, age_cols_10km_10er,
                                 'Insgesamt_Bevoelkerung_Zensus2022_Alter_in_10er-Jahresgruppen_10km-Gitter',
                                 reference_distribution_10km_10er)
ten_km_df = mix_age_distribution(ten_km_df, age_cols_10km_5er,
                                 "Insgesamt_Bevoelkerung_Zensus2022_Alter_in_5_Altersklassen_10km-Gitter",
                                 reference_distribution_10km_5er)
ten_km_df = ipf_adjustment(ten_km_df, row_totals_10er, col_totals_10er, age_cols_10km_10er)
ten_km_df = ipf_adjustment(ten_km_df, row_totals_5er, col_totals_5er, age_cols_10km_5er)
ten_km_df.to_csv("3_4adjusted_10km_ipf.csv", index=False)

# Do NOT integerize
# Save
print("Saved adjusted 10km IPF results to 3_4adjusted_10km_ipf.csv")
print("Col totals differ a tiny bit between 10er and 5er groups, but that's ok and left that way so we keep the best possible distribution."
      "The total population is the same for both groups, and the row totals match exactly.")


Adjusting age distribution at the 10km level...
Total nans: 208
Nans set to 0.
Diff in totals (should be 0: 0.0
Adjusted 52 cells.
Total cells: 3824
Adjusted 52 cells.
Total cells: 3824
82711150.0
82711150.0
82711150.0
82711150.0
82711150.0
82711150.0
82711150.0
82711150.0
82711150.0
82711150.0
82710877.0
82710877.0
82710877.0
82710877.0
82710877.0
82710877.0
82710877.0
82710877.0
82710877.0
82710877.0
Saved adjusted 10km IPF results to 3_4adjusted_10km_ipf.csv


In [8]:
# Check the results
import pandas as pd
import numpy as np

adj_df = pd.read_csv("3_4adjusted_10km_ipf.csv")
orig_df = pd.read_csv(r"C:\Users\petre\Documents\GitHub\MATSimPipeline\data\syn_pop\merged_10km_gitter.csv")

diff_df = adj_df[age_cols_10km_10er+age_cols_10km_5er] - orig_df[age_cols_10km_10er+age_cols_10km_5er]
print(diff_df.abs().max())

# Zeige jeweils das maximum der Col an
print(orig_df[age_cols_10km_10er+age_cols_10km_5er].max())

# Print the diffs between row totals (should be 0) the diffs between col totals (can be a slight diff) and the diff betw row and col totals for each grouping
print(f"Row total diff 10er/5er: {adj_df['Insgesamt_Bevoelkerung_Zensus2022_Alter_in_10er-Jahresgruppen_10km-Gitter'].sum() - adj_df['Insgesamt_Bevoelkerung_Zensus2022_Alter_in_5_Altersklassen_10km-Gitter'].sum()}")
print(f"Col total diff 10er/5er: {adj_df[age_cols_10km_10er].sum().sum() - adj_df[age_cols_10km_5er].sum().sum()}")
print(f"Row v Col total diff 10er: {adj_df['Insgesamt_Bevoelkerung_Zensus2022_Alter_in_10er-Jahresgruppen_10km-Gitter'].sum() - adj_df[age_cols_10km_10er].sum().sum()}")
print(f"Row v Col total diff 5er: {adj_df['Insgesamt_Bevoelkerung_Zensus2022_Alter_in_5_Altersklassen_10km-Gitter'].sum() - adj_df[age_cols_10km_5er].sum().sum()}")


Unter10_Zensus2022_Alter_in_10er-Jahresgruppen_10km-Gitter         3.566970
a10bis19_Zensus2022_Alter_in_10er-Jahresgruppen_10km-Gitter        4.596714
a20bis29_Zensus2022_Alter_in_10er-Jahresgruppen_10km-Gitter        6.695852
a30bis39_Zensus2022_Alter_in_10er-Jahresgruppen_10km-Gitter        2.985058
a40bis49_Zensus2022_Alter_in_10er-Jahresgruppen_10km-Gitter        4.232525
a50bis59_Zensus2022_Alter_in_10er-Jahresgruppen_10km-Gitter        7.174324
a60bis69_Zensus2022_Alter_in_10er-Jahresgruppen_10km-Gitter        8.726717
a70bis79_Zensus2022_Alter_in_10er-Jahresgruppen_10km-Gitter        6.543100
a80undaelter_Zensus2022_Alter_in_10er-Jahresgruppen_10km-Gitter    3.583652
Unter18_Zensus2022_Alter_in_5_Altersklassen_10km-Gitter            4.886568
a18bis29_Zensus2022_Alter_in_5_Altersklassen_10km-Gitter           8.151290
a30bis49_Zensus2022_Alter_in_5_Altersklassen_10km-Gitter           6.287109
a50bis64_Zensus2022_Alter_in_5_Altersklassen_10km-Gitter           7.563742
a65undaelter

In [1]:
# We now have good age distros for all 10km cells.
# Now, for 1km and 100km, mix age distributions for low pop cells downward from higher level, then do IPF

import pandas as pd
import numpy as np
import re
from tqdm import tqdm


def smart_integerize(df, pop_cols):
    for pop_col in pop_cols:
        total_before = df[pop_col].sum()
        df[pop_col] = df[pop_col].round().astype(int)
        total_after = df[pop_col].sum()
        diff = total_before - total_after
        if diff != 0:
            sorted_indices = df[pop_col].abs().sort_values(ascending=False).index[:abs(int(diff))]
            df.loc[sorted_indices, pop_col] += int(diff / abs(diff))
    return df


def ipf_adjustment(df, row_control_totals, col_control_totals, relevant_cols, max_iterations=10, tol=1e-6):
    for _ in range(max_iterations):

    # Here we do rows last, so the population total per cell is hit exactly

        col_totals = df[relevant_cols].sum()
        scaling_factors = col_control_totals / col_totals
        # Set inf to 1
        scaling_factors[~np.isfinite(scaling_factors)] = 1
        df[relevant_cols] = df[relevant_cols].multiply(scaling_factors, axis=1)
    
        row_totals = df[relevant_cols].sum(axis=1)
        scaling_factors = row_control_totals / row_totals
        # Set inf to 1
        scaling_factors[~np.isfinite(scaling_factors)] = 1
        df[relevant_cols] = df[relevant_cols].multiply(scaling_factors, axis=0)

        error = 1
        if error < tol:
            break
    return df


def mix_distributions(df, age_cols, total_col, reference_cols, threshold=100):
    mask = (df[total_col] > 0) & (df[total_col] < threshold)
    cell_count = mask.sum()

    if cell_count == 0:
        print("No cells adjusted.")
        return df

    weight = df.loc[mask, total_col] / threshold
    weight = weight.values.reshape(-1, 1)

    pop = df.loc[mask, total_col].values.reshape(-1, 1)
    age_data = df.loc[mask, age_cols].values
    # Fit age data
    age_data = age_data / age_data.sum(axis=1).reshape(-1, 1) * pop  # scale age data to pop
    #age_data = np.nan_to_num(age_data)
    ref_data = df.loc[mask, reference_cols].values * pop  # scale reference by pop

    adjusted = weight * age_data + (1 - weight) * ref_data
    df.loc[mask, age_cols] = adjusted

    adjusted_row_sums = df.loc[mask, age_cols].sum(axis=1)
    print(f"Row sums mixing: {adjusted_row_sums.sum()}")
    print(f"Total pop: {df.loc[mask, total_col].sum()}")
    #assert np.allclose(adjusted_row_sums, df.loc[mask, total_col], atol=1e-3), "Row sums do not match total pop."
    
    print(f"Adjusted {cell_count} cells.")
    print(f"Total cells: {len(df)}")
    return df


# Load preprocessed data
one_km_df = pd.read_csv("3adjusted_1km_with_higher.csv")
hundred_m_df = pd.read_csv("3adjusted_100m_with_higher.csv")
ten_km_df = pd.read_csv("3_4adjusted_10km_ipf.csv")

age_cols_10km_10er = [
    # 10er-Gruppen
    'Unter10_Zensus2022_Alter_in_10er-Jahresgruppen_10km-Gitter',
    'a10bis19_Zensus2022_Alter_in_10er-Jahresgruppen_10km-Gitter',
    'a20bis29_Zensus2022_Alter_in_10er-Jahresgruppen_10km-Gitter',
    'a30bis39_Zensus2022_Alter_in_10er-Jahresgruppen_10km-Gitter',
    'a40bis49_Zensus2022_Alter_in_10er-Jahresgruppen_10km-Gitter',
    'a50bis59_Zensus2022_Alter_in_10er-Jahresgruppen_10km-Gitter',
    'a60bis69_Zensus2022_Alter_in_10er-Jahresgruppen_10km-Gitter',
    'a70bis79_Zensus2022_Alter_in_10er-Jahresgruppen_10km-Gitter',
    'a80undaelter_Zensus2022_Alter_in_10er-Jahresgruppen_10km-Gitter']

age_cols_10km_5er = [
    # 5er-Gruppen
    'Unter18_Zensus2022_Alter_in_5_Altersklassen_10km-Gitter',
    'a18bis29_Zensus2022_Alter_in_5_Altersklassen_10km-Gitter',
    'a30bis49_Zensus2022_Alter_in_5_Altersklassen_10km-Gitter',
    'a50bis64_Zensus2022_Alter_in_5_Altersklassen_10km-Gitter',
    'a65undaelter_Zensus2022_Alter_in_5_Altersklassen_10km-Gitter'
]

# 1km level
print("Adjusting age distribution at the 1km level...")

age_cols_1km_5er = [col.replace('10km', '1km') for col in age_cols_10km_5er]
age_cols_1km_10er = [col.replace('10km', '1km') for col in age_cols_10km_10er]
# Count nans
nans = one_km_df[age_cols_1km_10er+age_cols_1km_5er].isna().sum().sum()
print(f"Total nans: {nans}")
# Set nans in given cols to 0.00001 (to avoid div by 0)
one_km_df[age_cols_1km_10er+age_cols_1km_5er] = one_km_df[age_cols_1km_10er+age_cols_1km_5er].fillna(0.00001)
print("Nans set to 0.00001.")

ten_km_df['age_sum_10er'] = ten_km_df[age_cols_10km_10er].sum(axis=1) # Minimal!! anders als die "Insgesamt" Reihensumme (nur bei 10km!), deswegen so
ten_km_df['age_sum_5er'] = ten_km_df[age_cols_10km_5er].sum(axis=1)

for col in age_cols_10km_10er:
    ten_km_df[f'{col}_prop'] = ten_km_df[col] / ten_km_df['age_sum_10er']
for col in age_cols_10km_5er:
    ten_km_df[f'{col}_prop'] = ten_km_df[col] / ten_km_df['age_sum_5er']

age_prop_cols_10er = [f'{col}_prop' for col in age_cols_10km_10er]
age_prop_cols_5er = [f'{col}_prop' for col in age_cols_10km_5er]

one_km_df = one_km_df.merge(ten_km_df[['GITTER_ID_10km'] + age_prop_cols_5er + age_prop_cols_10er],
                            on='GITTER_ID_10km', how='left')

assert one_km_df['Insgesamt_Bevoelkerung_Zensus2022_Alter_in_10er-Jahresgruppen_1km-Gitter'].sum() == one_km_df[
    'Insgesamt_Bevoelkerung_Zensus2022_Alter_in_5_Altersklassen_1km-Gitter'].sum()

# We now have the relative distributions of the 10km cells in the 1km cells. Now they are mixed where needed.
one_km_df = mix_distributions(one_km_df, age_cols_1km_10er,
                                 'Insgesamt_Bevoelkerung_Zensus2022_Alter_in_10er-Jahresgruppen_1km-Gitter', age_prop_cols_10er)
one_km_df = mix_distributions(one_km_df, age_cols_1km_5er,
                                    'Insgesamt_Bevoelkerung_Zensus2022_Alter_in_5_Altersklassen_1km-Gitter', age_prop_cols_5er)

# Per-cell IPF: We have mixed distributions per 1km cell. To make sure the 1km cell distros also in sum match the 10km cells, AND
# the population totals per cell are still met (which they already are but adjusting distro messes with them) we do IPF, but per 10km cell.

grouped_1km = one_km_df.groupby('GITTER_ID_10km')
debug_counter = 0
for gid, group in tqdm(grouped_1km):
    debug_counter += 1
    # Only ipf 
    row_control_totals_10er = group['Insgesamt_Bevoelkerung_Zensus2022_Alter_in_10er-Jahresgruppen_1km-Gitter'].values.copy()
    # Get col totals from 10km cell
    col_control_totals_10er = ten_km_df.loc[ten_km_df['GITTER_ID_10km'] == gid, age_cols_10km_10er].values[0].copy()
    relevant_cols = age_cols_1km_10er
    group = ipf_adjustment(group, row_control_totals_10er, col_control_totals_10er, relevant_cols)
    one_km_df.loc[group.index, relevant_cols] = group[relevant_cols]

    row_control_totals_5er = group['Insgesamt_Bevoelkerung_Zensus2022_Alter_in_5_Altersklassen_1km-Gitter'].values[0].copy()
    # Get col totals from 10km cell
    col_control_totals_5er = ten_km_df.loc[ten_km_df['GITTER_ID_10km'] == gid, age_cols_10km_5er].values[0].copy()
    relevant_cols = age_cols_1km_5er
    group = ipf_adjustment(group, row_control_totals_5er, col_control_totals_5er, relevant_cols)
    one_km_df.loc[group.index, relevant_cols] = group[relevant_cols]
    if debug_counter > 20:
        break
    
# As result, we have the robust distros from more aggregated cells, but localized data mixed in. 
# NO integerization

# Save only top 100 rows
#one_km_df.head(100).to_csv("4adjusted_1km_mixing_ipf.csv", index=False)

one_km_df.to_csv("4adjusted_1km_mixing_ipf.csv", index=False)
print("Saved adjusted 1km IPF results to 4adjusted_1km_mixing_ipf.csv")

# Short check that the totals were not messed up (we don't edit them here but just to be sure)
print("Totals (should match):")
print(one_km_df['Insgesamt_Bevoelkerung_Zensus2022_Alter_in_10er-Jahresgruppen_1km-Gitter'].sum())
print(one_km_df['Insgesamt_Bevoelkerung_Zensus2022_Alter_in_5_Altersklassen_1km-Gitter'].sum())
print(ten_km_df['Insgesamt_Bevoelkerung_Zensus2022_Alter_in_10er-Jahresgruppen_10km-Gitter'].sum())
print(ten_km_df['Insgesamt_Bevoelkerung_Zensus2022_Alter_in_5_Altersklassen_10km-Gitter'].sum())

totals_diffs = (ten_km_df['Insgesamt_Bevoelkerung_Zensus2022_Alter_in_10er-Jahresgruppen_10km-Gitter'].values -
                one_km_df.groupby('GITTER_ID_10km')['Insgesamt_Bevoelkerung_Zensus2022_Alter_in_10er-Jahresgruppen_1km-Gitter'].sum().values)
print(max(totals_diffs))
totals_diffs = (ten_km_df['Insgesamt_Bevoelkerung_Zensus2022_Alter_in_5_Altersklassen_10km-Gitter'].values -
                one_km_df.groupby('GITTER_ID_10km')['Insgesamt_Bevoelkerung_Zensus2022_Alter_in_5_Altersklassen_1km-Gitter'].sum().values)
print(max(totals_diffs))



# -------------------------------------------------------------------------------------------
# 100m level with integerization
print("Adjusting age distribution at the 100m level...")
age_cols_100m_5er = [
    # 5er-Gruppen
    'Unter18_Zensus2022_Alter_5Altersklassen_100m-Gitter',
    'a18bis29_Zensus2022_Alter_5Altersklassen_100m-Gitter',
    'a30bis49_Zensus2022_Alter_5Altersklassen_100m-Gitter',
    'a50bis64_Zensus2022_Alter_5Altersklassen_100m-Gitter',
    'a65undaelter_Zensus2022_Alter_5Altersklassen_100m-Gitter'
]
# age_cols_1km_5er = [
#     # 5er-Gruppen
#     'Unter18_Zensus2022_Alter_5Altersklassen_1km-Gitter',
#     'a18bis29_Zensus2022_Alter_5Altersklassen_1km-Gitter',
#     'a30bis49_Zensus2022_Alter_5Altersklassen_1km-Gitter',
#     'a50bis64_Zensus2022_Alter_5Altersklassen_1km-Gitter',
#     'a65undaelter_Zensus2022_Alter_5Altersklassen_1km-Gitter'
# ]
#age_cols_100m_5er = [col.replace('10km', '100m') for col in age_cols_10km_5er]
age_cols_100m_10er = [col.replace('10km', '100m') for col in age_cols_10km_10er]
# Count nans
nans = hundred_m_df[age_cols_100m_5er+age_cols_100m_10er].isna().sum().sum()
print(f"Total nans: {nans}")
# Set nans in given cols to 0
hundred_m_df[age_cols_100m_5er+age_cols_100m_10er] = hundred_m_df[age_cols_100m_5er+age_cols_100m_10er].fillna(0.00001)
print("Nans set to 0.00001.")

#TODO: IPF is broken thus the prop cols dont add up to 1 and make 100m bad too
for col in age_cols_1km_10er:
    one_km_df[f'{col}_prop'] = one_km_df[col] / one_km_df['Insgesamt_Bevoelkerung_Zensus2022_Alter_in_10er-Jahresgruppen_1km-Gitter']
for col in age_cols_1km_5er:
    one_km_df[f'{col}_prop'] = one_km_df[col] / one_km_df['Insgesamt_Bevoelkerung_Zensus2022_Alter_in_5_Altersklassen_1km-Gitter']
age_prop_cols_10er = [f'{col}_prop' for col in age_cols_1km_10er]
age_prop_cols_5er = [f'{col}_prop' for col in age_cols_1km_5er]

hundred_m_df = hundred_m_df.merge(one_km_df[['GITTER_ID_1km'] + age_prop_cols_5er + age_prop_cols_10er],
                            on='GITTER_ID_1km', how='left')

assert hundred_m_df['Insgesamt_Bevoelkerung_Zensus2022_Alter_in_10er-Jahresgruppen_100m-Gitter'].sum() == hundred_m_df[
    'Insgesamt_Bevoelkerung_Zensus2022_Alter_in_5_Altersklassen_100m-Gitter'].sum()

# We now have the relative distributions of the 1km cells in the 100m cells. Now they are mixed where needed (which is almost always at this level)

#age_cols_100m_10er = [col.replace('10km', '1km') for col in age_cols_10km_10er]
#age_cols_1km_5er = [col.replace('10km', '1km') for col in age_cols_10km_5er]

hundred_m_df = mix_distributions(hundred_m_df, age_cols_100m_10er,
                                 'Insgesamt_Bevoelkerung_Zensus2022_Alter_in_10er-Jahresgruppen_100m-Gitter', age_prop_cols_10er)
hundred_m_df = mix_distributions(hundred_m_df, age_cols_100m_5er,
                                    'Insgesamt_Bevoelkerung_Zensus2022_Alter_in_5_Altersklassen_100m-Gitter', age_prop_cols_5er)


# Per-cell IPF: We have mixed distributions per 1km cell. To make sure the 1km cell distros also in sum match the 10km cells, AND
# the population totals per cell are still met (which they already are but adjusting distro messes with them) we do IPF, but per 10km cell.

grouped_100m = hundred_m_df.groupby('GITTER_ID_1km')
debug_counter = 0
for gid, group in tqdm(grouped_100m):
    debug_counter += 1
    row_control_totals_10er = group['Insgesamt_Bevoelkerung_Zensus2022_Alter_in_10er-Jahresgruppen_100m-Gitter'].values[0].copy()
    # Get col totals from 1km cell
    col_control_totals_10er = one_km_df.loc[one_km_df['GITTER_ID_1km'] == gid, age_cols_1km_10er].values[0].copy()
    relevant_cols = age_cols_100m_10er
    group = ipf_adjustment(group, row_control_totals_10er, col_control_totals_10er, relevant_cols)
    hundred_m_df.loc[group.index, relevant_cols] = group[relevant_cols]

    row_control_totals_5er = group['Insgesamt_Bevoelkerung_Zensus2022_Alter_in_5_Altersklassen_100m-Gitter'].values[0].copy()
    # Get col totals from 1km cell
    col_control_totals_5er = one_km_df.loc[one_km_df['GITTER_ID_1km'] == gid, age_cols_1km_5er].values[0].copy()
    relevant_cols = age_cols_100m_5er
    group = ipf_adjustment(group, row_control_totals_5er, col_control_totals_5er, relevant_cols)
    hundred_m_df.loc[group.index, relevant_cols] = group[relevant_cols]
    if debug_counter > 20:
        break
    
# As result, we have the robust distros from more aggregated cells, but localized data mixed in. 
# # Integerize (not for now while debugging)
hundred_m_df.to_csv("4adjusted_100m_mixing_ipf.csv", index=False)
print("Saved adjusted 1km IPF results to 4adjusted_1km_mixing_ipf.csv")


C:\Users\petre\AppData\Local\Temp\ipykernel_1568\466555968.py:77: DtypeWarning: Columns (34) have mixed types. Specify dtype option on import or set low_memory=False.
  one_km_df = pd.read_csv("3adjusted_1km_with_higher.csv")
C:\Users\petre\AppData\Local\Temp\ipykernel_1568\466555968.py:78: DtypeWarning: Columns (27,34,36) have mixed types. Specify dtype option on import or set low_memory=False.
  hundred_m_df = pd.read_csv("3adjusted_100m_with_higher.csv")


Adjusting age distribution at the 1km level...
Total nans: 719488
Nans set to 0.00001.
Row sums mixing: 3676946.0
Total pop: 3676946
Adjusted 122315 cells.
Total cells: 212746
Row sums mixing: 3676946.0
Total pop: 3676946
Adjusted 122315 cells.
Total cells: 212746


  1%|          | 20/3824 [00:11<36:35,  1.73it/s]


Saved adjusted 1km IPF results to 4adjusted_1km_mixing_ipf.csv
Totals (should match):
82711381
82711381
82711382.0
82711382.0
18.0
18.0
Adjusting age distribution at the 100m level...
Total nans: 21842251
Nans set to 0.00001.
Row sums mixing: 62775848.737544045
Total pop: 62778168
Adjusted 2963028 cells.
Total cells: 3147936
Row sums mixing: 62728384.679754764
Total pop: 62778168
Adjusted 2963028 cells.
Total cells: 3147936


Error evaluating: thread_id: pid_1568_id_2080410978448
frame_id: 2080753941104
scope: FRAME
attrs: hundred_m_df
Traceback (most recent call last):
  File "C:\Users\petre\AppData\Local\JetBrains\PyCharm 2024.1.2\plugins\python-ce\helpers\pydev\_pydevd_bundle\pydevd_resolver.py", line 178, in _getPyDictionary
    attr = getattr(var, n)
           ^^^^^^^^^^^^^^^
  File "C:\Users\petre\AppData\Local\miniforge3\envs\MATSimPipeline\Lib\site-packages\pandas\core\frame.py", line 3978, in T
    return self.transpose()
           ^^^^^^^^^^^^^^^^
  File "C:\Users\petre\AppData\Local\miniforge3\envs\MATSimPipeline\Lib\site-packages\pandas\core\frame.py", line 3937, in transpose
    new_arr = self.values.T
              ^^^^^^^^^^^
  File "C:\Users\petre\AppData\Local\miniforge3\envs\MATSimPipeline\Lib\site-packages\pandas\core\frame.py", line 12664, in values
    return self._mgr.as_array()
           ^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\petre\AppData\Local\miniforge3\envs\MATSimPipeline\Lib\si

Saved adjusted 1km IPF results to 4adjusted_1km_mixing_ipf.csv


In [ ]:
 # Check results
one_km_df = pd.read_csv("4adjusted_1km_mixing_ipf.csv")
ten_km_df = pd.read_csv("3_4adjusted_10km_ipf.csv")
hundred_m_df = pd.read_csv("4adjusted_100m_ipf.csv")

# Short check that the totals were not messed up (we don't edit them here but just to be sure)
print("Totals (should match):")
print(one_km_df['Insgesamt_Bevoelkerung_Zensus2022_Alter_in_10er-Jahresgruppen_1km-Gitter'].sum())
print(one_km_df['Insgesamt_Bevoelkerung_Zensus2022_Alter_in_5_Altersklassen_1km-Gitter'].sum())
print(ten_km_df['Insgesamt_Bevoelkerung_Zensus2022_Alter_in_10er-Jahresgruppen_10km-Gitter'].sum())
print(ten_km_df['Insgesamt_Bevoelkerung_Zensus2022_Alter_in_5_Altersklassen_10km-Gitter'].sum())

totals_diffs = (ten_km_df['Insgesamt_Bevoelkerung_Zensus2022_Alter_in_10er-Jahresgruppen_10km-Gitter'].values -
                one_km_df.groupby('GITTER_ID_10km')['Insgesamt_Bevoelkerung_Zensus2022_Alter_in_10er-Jahresgruppen_1km-Gitter'].sum().valus())
print(max(totals_diffs))
totals_diffs = (ten_km_df['Insgesamt_Bevoelkerung_Zensus2022_Alter_in_5_Altersklassen_10km-Gitter'].values -
                one_km_df.groupby('GITTER_ID_10km')['Insgesamt_Bevoelkerung_Zensus2022_Alter_in_5_Altersklassen_1km-Gitter'].sum().values)
print(max(totals_diffs))

# Compare total distros
distro_diffs = (ten_km_df[age_cols_10km_10er].sum() - one_km_df[age_cols_1km_10er].sum()).abs()
print(distro_diffs.max())
distro_diffs = (ten_km_df[age_cols_10km_5er].sum() - one_km_df[age_cols_1km_5er].sum()).abs()
print(distro_diffs.max())
distro_diffs = (one_km_df[age_cols_1km_10er].sum() - hundred_m_df[age_cols_100m_10er].sum()).abs()
print(distro_diffs.max())
distro_diffs = (one_km_df[age_cols_1km_5er].sum() - hundred_m_df[age_cols_100m_5er].sum()).abs()
print(distro_diffs.max())

# Compare per-cell distros


